## I. Setup Software and Some Libraries

We first need to set the working environment and the path to the dataset.

In [739]:
import sys

SOFTWARE_DIR = '/app/data/spine/' # Change this path to your software install
DATA_DIR = '/app/data/' # Change this path if you are not on SDF (see main README)

# Set software directory
sys.path.append(SOFTWARE_DIR)

import numpy as np
import math
import pandas as pd
from collections import OrderedDict

from scipy.spatial.distance import cdist

Now pass the analysis configuration.

In [740]:
import yaml
from spine.driver import Driver

# file_name = 'latest_packet-0050017-2024_07_09_00_04_33_CDT.LARCV_spine.h5' # latest spine reprocessing

# file_name = 'packet-0050017-2024_07_09_00_04_33_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_00_14_34_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_00_24_35_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_00_34_36_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_00_44_37_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_00_54_38_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_01_04_39_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_01_14_40_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_01_24_41_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_01_34_42_CDT.SPINE.h5' # old spine processing
# file_name = 'packet-0050017-2024_07_09_01_44_44_CDT.SPINE.h5' # old spine processing
file_name = 'packet-0050017-2024_07_09_01_54_45_CDT.SPINE.h5' # old spine processing


DATA_PATH = DATA_DIR + file_name

cfg = '''
base:
  iterations: 10
io:
  reader:
    name: hdf5 # Type of input reader
    file_keys: DATA_PATH # Path to the data file
    create_run_map: true # Creates a map between run/event pairs and entry
    skip_unknown_attrs: true # Allows to load files made with an older SPINE
build:
  mode: reco # Build only reconstructed objects (no truth information)
  fragments: false # Do not build fragment objects (not stored by default)
  particles: true # Build `RecoParticle`/`TruthParticle` objects
  interactions: true # Build `RecoInteraction`/`TruthInteraction` objects

'''.replace('DATA_PATH', file_name)

cfg = yaml.safe_load(cfg)
driver = Driver(cfg)



 ██████████   ██████████    ███   ███       ██   ███████████
███        █  ██       ███   █    █████     ██   ██         
  ████████    ██       ███  ███   ██  ████  ██   ██████████ 
█        ███  ██████████     █    ██     █████   ██         
 ██████████   ██            ███   ██       ███   ███████████

Release version: 0.2.2

$CUDA_VISIBLE_DEVICES=

Configuration processed at: Linux c033da5ae0da 5.15.167.4-microsoft-standard-WSL2 #1 SMP Tue Nov 5 00:21:55 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux

base: {iterations: 10, seed: 1737428287}
io:
  reader: {name: hdf5, file_keys: packet-0050017-2024_07_09_01_54_45_CDT.SPINE.h5,
    create_run_map: true, skip_unknown_attrs: true}
build: {mode: reco, fragments: false, particles: true, interactions: true}

Will load 1 file(s):
  - packet-0050017-2024_07_09_01_54_45_CDT.SPINE.h5

Total number of entries in the file(s): 456

Total number of entries selected: 456



Map ND-LAr events to their corresponding `entry` numbers in SPINE files using the `run_info` field.

In [741]:
print('Number of entries in the loader:', len(driver))

import h5py

# Target Event (Update this as needed)
target_event = 171 # Example: Target Event Number

# Initialize entry as None
ENTRY = None

# Locate Entry Corresponding to Target Event
with h5py.File(DATA_PATH, 'r') as f:
    run_info = f['run_info']
    for idx, (run, subrun, event) in enumerate(run_info):
        if event == target_event:
            ENTRY = idx
            print(f"Found target event {target_event} at entry {idx}")
            break

if ENTRY is None:
    print(f"Target event {target_event} not found.")

Number of entries in the loader: 456
Found target event 171 at entry 171


Now initialize the driver with this configuration.

In [742]:
if ENTRY is not None:
    data = driver.process(entry=ENTRY)
    print(f"Processing data for entry: {ENTRY}")
else:
    print("No valid entry found. Ensure Cell 1 has run successfully.")

Processing data for entry: 171


Retrieves and analyzes the `reco_particles` and `reco_interactions` data structures. It provides a count of each structure and collects the unique **Reco Interaction IDs** from the `reco_particles` dataset.

In [743]:
# Retrieving data structures
reco_particles = data['reco_particles']
reco_interactions = data['reco_interactions']

# Print counts of different data structures
print(f"Reco Particles: {len(reco_particles)}")
print(f"Reco Interactions: {len(reco_interactions)}")

# Collect Reco and Truth Interaction IDs
reco_interaction_ids = set(particle.interaction_id for particle in reco_particles)

print(f"Reco Interaction IDs: {reco_interaction_ids}")  # Sorted for better readability

# Count the number of particles associated with each interaction ID
interaction_particle_counts = {}
for particle in reco_particles:
    interaction_id = particle.interaction_id
    if interaction_id not in interaction_particle_counts:
        interaction_particle_counts[interaction_id] = 0
    interaction_particle_counts[interaction_id] += 1

# Print the number of particles for each interaction ID
print("\nNumber of particles associated with each interaction ID:")
for interaction_id, count in interaction_particle_counts.items():
    print(f"Interaction ID {interaction_id}: {count} particles")

Reco Particles: 11
Reco Interactions: 4
Reco Interaction IDs: {0, 1, 2, 3}

Number of particles associated with each interaction ID:
Interaction ID 0: 3 particles
Interaction ID 1: 1 particles
Interaction ID 2: 3 particles
Interaction ID 3: 4 particles


## II. All Reconstructed Particles for Target Interaction ID

The script locates the target reconstructed interaction ID (`target_reco_interaction_id`) in the dataset, retrieves its entry, and filters all **reconstructed particles** that belong to the specified interaction ID.

It prints details of reconstructed particles and their interaction IDs, along with the total count of filtered particles.

In [744]:
import h5py

# Define the target reconstructed interaction ID
target_reco_interaction_id = 3 # Example: Target Interaction ID

# Locate the entry corresponding to the target event
with h5py.File(DATA_PATH, 'r') as f:
    run_info = f['run_info']
    target_entry = None
    
    for idx, (run, subrun, event) in enumerate(run_info):
        if event == target_event:
            target_entry = idx
            print(f"Found target event {target_event} at entry {idx}")
            break
    else:
        print(f"Event {target_event} not found in run_info.")
        target_entry = None

# Process the entry if found
if target_entry is not None:
    data = driver.process(entry=target_entry)
    reco_particles = data['reco_particles']
    print(f"Total Reconstructed Particles for Event {target_event}: {len(reco_particles)}\n")
    
    # Filter particles belonging to the target interaction ID
    all_reco_particles = []
    for i, particle in enumerate(reco_particles):
        interaction_id = particle.interaction_id  # Get the interaction ID
        
        # Apply target interaction ID filter
        if interaction_id == target_reco_interaction_id:
            all_reco_particles.append(particle)
            print(f"reco_particles[{i}]: {particle} | Reco Interaction ID: {interaction_id}\n")
    
    print(f"Total Reconstructed Particles for Interaction ID {target_reco_interaction_id}: {len(all_reco_particles)}")

Found target event 171 at entry 171
Total Reconstructed Particles for Event 171: 11

reco_particles[6]: RecoParticle(ID: 6   | PID: Electron | Primary: 0  | Size: 10    | Match: -1 ) | Reco Interaction ID: 3

reco_particles[8]: RecoParticle(ID: 8   | PID: Muon     | Primary: 1  | Size: 143   | Match: -1 ) | Reco Interaction ID: 3

reco_particles[9]: RecoParticle(ID: 9   | PID: Proton   | Primary: 0  | Size: 80    | Match: -1 ) | Reco Interaction ID: 3

reco_particles[10]: RecoParticle(ID: 10  | PID: Kaon     | Primary: 1  | Size: 80    | Match: -1 ) | Reco Interaction ID: 3

Total Reconstructed Particles for Interaction ID 3: 4


In [745]:
from spine.vis import Drawer
from spine.vis import GeoDrawer, scatter_points, layout3d
from plotly import graph_objs as go
import plotly.io as pio
import os

# Ensure all_reco_particles from Cell 1 is available
if not all_reco_particles:
    raise ValueError("No filtered particles found in Cell 1. Please run Cell 1 first.")

# Extract IDs of the filtered particles from Cell 1
filtered_particle_ids = [particle.id for particle in all_reco_particles]

# Pre-filter the data to include only relevant reconstructed particles
data['reco_particles'] = [p for p in data['reco_particles'] if p.id in filtered_particle_ids]

# Extract the specific part ("00_04_33") from the filename
file_segment = '_'.join(os.path.basename(DATA_PATH).split('_')[3:6])

# Create the images directory if it doesn't exist
output_dir = "images"
os.makedirs(output_dir, exist_ok=True)

# Initialize the drawer with filtered data
drawer = Drawer(data, draw_mode='reco', detector='2x2')

# Draw a plot of the particle instances based on the filtered particles
print('\n>>> All Particles <<<')
fig_particles = drawer.get(
    'particles',
    attr='pid',
    draw_end_points=True,
    draw_vertices=True,
    split_traces=True  # Ensure each particle gets a separate legend entry
)

# Save the filtered particle plot as HTML and PNG in the images folder
html_filename = os.path.join(output_dir, f"sandbox_{file_segment}_entry_{target_entry}_event_{target_event}_int_{target_reco_interaction_id}_particles_all.html")
png_filename = os.path.join(output_dir, f"sandbox_{file_segment}_entry_{target_entry}_event_{target_event}_int_{target_reco_interaction_id}_particles_all.png")

# Save as HTML
pio.write_html(fig_particles, file=html_filename)
print(f"Filtered Particle Plot saved as HTML: {html_filename}")

# Save as PNG
pio.write_image(fig_particles, file=png_filename, format='png', width=1000, height=800)
print(f"Filtered Particle Plot saved as PNG: {png_filename}")

# Display the visualization
fig_particles.show()



>>> All Particles <<<
Filtered Particle Plot saved as HTML: images/sandbox_01_54_45_entry_171_event_171_int_3_particles_all.html
Filtered Particle Plot saved as PNG: images/sandbox_01_54_45_entry_171_event_171_int_3_particles_all.png


### III. Filtering Reconstructed Particles for Target Interaction ID Within Spatial Limits

The script locates the target reconstructed interaction ID (`target_reco_interaction_id`) in the dataset, retrieves its entry, and filters **primary reconstructed particles** that:
- Belong to the specified interaction ID.
- Are `primary`.
- Are within defined spatial limits (`minX`, `maxX`, `minY`, `maxY`, `minZ`, `maxZ`).
- A `length` selection can also be applied and printed.

It prints details of reconstructed particles and their interaction IDs, along with the total count of filtered particles.

In [746]:
import h5py

# Define the target reconstructed interaction ID
# target_reco_interaction_id = 0  # Example: Target Interaction ID

# Define spatial limits
distance_from_wall = 0
minX = -63.931 + distance_from_wall
maxX = +63.931 - distance_from_wall
minY = -62.076 + distance_from_wall
maxY = +62.076 - distance_from_wall
minZ = -64.538 + distance_from_wall
maxZ = +64.538 - distance_from_wall

# Minimum length cut in cm
length_cut = 2.0

# Locate the entry corresponding to the target event
with h5py.File(DATA_PATH, 'r') as f:
    run_info = f['run_info']
    target_entry = None
    
    for idx, (run, subrun, event) in enumerate(run_info):
        if event == target_event:
            target_entry = idx
            print(f"Found target event {target_event} at entry {idx}")
            break
    else:
        print(f"Event {target_event} not found in run_info.")
        target_entry = None

# Process the entry if found
if target_entry is not None:
    data = driver.process(entry=target_entry)
    reco_particles = data['reco_particles']
    print(f"Total Reconstructed Particles for Event {target_event}: {len(reco_particles)}")
    
    # Filter and print particles within limits, primary, and belonging to the target interaction ID
    filtered_primary_particles = []
    for i, particle in enumerate(reco_particles):
        # Access start_point for spatial coordinates using dot notation
        start_point = particle.start_point
        x, y, z = start_point
        length = particle.length  # Length attribute of the particle

        
        # Check if the particle is primary and matches the target interaction ID
        is_primary = particle.is_primary
        interaction_id = particle.interaction_id  # Get the interaction ID
        
        # Apply spatial limits, primary condition, and target interaction ID
        if (
            length > length_cut and
            is_primary and
            minX < x < maxX and
            minY < y < maxY and
            minZ < z < maxZ and
            interaction_id == target_reco_interaction_id
        ):
            filtered_primary_particles.append(particle)
            # print(f"reco_particles[{i}]: {particle} | Reco Interaction ID: {interaction_id}\n")
            print(f"reco_particles[{i}]: {particle} | Reco Interaction ID: {interaction_id} | Length: {length:.2f} cm\n")

    
    print(f"Total Reconstructed Particles Within Limits for Interaction ID {target_reco_interaction_id}: {len(filtered_primary_particles)}")

Found target event 171 at entry 171
Total Reconstructed Particles for Event 171: 11
reco_particles[8]: RecoParticle(ID: 8   | PID: Muon     | Primary: 1  | Size: 143   | Match: -1 ) | Reco Interaction ID: 3 | Length: 34.01 cm

reco_particles[10]: RecoParticle(ID: 10  | PID: Kaon     | Primary: 1  | Size: 80    | Match: -1 ) | Reco Interaction ID: 3 | Length: 18.22 cm

Total Reconstructed Particles Within Limits for Interaction ID 3: 2


### IV. Filtered Particle Visualization

This script visualizes and saves plots of filtered reconstructed particles and interactions. 
   - Particle plots are saved as:
     - **HTML**: `sandbox_<file_segment>_entry_<ENTRY>_event_<target_event>_int_<target_reco_interaction_id>_particles.html`
     - **PNG**: `sandbox_<file_segment>_entry_<ENTRY>_event_<target_event>_int_<target_reco_interaction_id>_particles.png`

In [747]:
from spine.vis import Drawer
from spine.vis import GeoDrawer, scatter_points, layout3d
from plotly import graph_objs as go
import plotly.io as pio
import os

# Extract IDs of the filtered particles
filtered_particle_ids = [particle.id for particle in filtered_primary_particles]

# Pre-filter the data to include only relevant reconstructed particles
data['reco_particles'] = [p for p in data['reco_particles'] if p.id in filtered_particle_ids]

# Extract the specific part ("00_04_33") from the filename
file_segment = '_'.join(os.path.basename(DATA_PATH).split('_')[3:6])

# Create the images directory if it doesn't exist
output_dir = "images"
os.makedirs(output_dir, exist_ok=True)

# Initialize the drawer with filtered data
drawer = Drawer(data, draw_mode='reco', detector='2x2')

# Draw a plot of the raw image (from flow)
geo_drawer = GeoDrawer('2x2')
graph = scatter_points(data['points'], color=data['depositions'], colorscale='Inferno', name='Depositions')
graph += geo_drawer.tpc_traces()

# Draw a plot of the particle instances based on the filtered particles
print('\n>>> Selected Particles <<<')
# fig_particles = drawer.get('particles', 'pid', draw_end_points=True, draw_vertices=True)
fig_particles = drawer.get(
    'particles',
    attr='pid',
    draw_end_points=True,
    draw_vertices=True,
    split_traces=True  # Ensure each particle gets a separate legend entry
)

# Save the filtered particle plot as HTML and PNG in the images folder
html_filename = os.path.join(output_dir, f"sandbox_{file_segment}_entry_{ENTRY}_event_{target_event}_int_{target_reco_interaction_id}_particles_selected.html")
png_filename = os.path.join(output_dir, f"sandbox_{file_segment}_entry_{ENTRY}_event_{target_event}_int_{target_reco_interaction_id}_particles_selected.png")

# Save as HTML
pio.write_html(fig_particles, file=html_filename)
print(f"Filtered Particle Plot saved as HTML: {html_filename}")

# Save as PNG
pio.write_image(fig_particles, file=png_filename, format='png', width=1000, height=800)
print(f"Filtered Particle Plot saved as PNG: {png_filename}")

# Display the visualization
fig_particles.show()

# If interaction filtering is required, modify and draw interactions
filtered_interactions = [
    interaction for interaction in data['reco_interactions']
    if any(particle.id in filtered_particle_ids for particle in interaction.particles)
]
data['reco_interactions'] = filtered_interactions



>>> Selected Particles <<<
Filtered Particle Plot saved as HTML: images/sandbox_01_54_45_entry_171_event_171_int_3_particles_selected.html
Filtered Particle Plot saved as PNG: images/sandbox_01_54_45_entry_171_event_171_int_3_particles_selected.png
